# Find candidate proteins for protein-ligand free energy benchmarks

Start with the protein-ligand validation sets in BindinDB, filter for:

- Small(ish)
- No metals
- No histidines near the ligand

Would also be nice to filter for surface exposed binding pocket, but not sure exactly how to do that yet.

In [2]:
import os
import urllib
import pandas as pd
import subprocess as sp

from io import StringIO


In [3]:
%load_ext blackcellmagic

## Read in the protein-ligand validation sets from BindingDB
http://bindingdb.org/validation_sets/index.jsp

In [4]:
validation_set = pd.read_csv("validation_sets_PDBs.tsv", names={"PDB"})


Extract just the PDBs from this list.

In [5]:
pdbs = pd.Series.tolist(validation_set["PDB"])

In [6]:
print(f"Starting with {len(pdbs)} structures...")

Starting with 778 structures...


## Filter for proteins less than 200 AAs

In [7]:
def filter_chain_length(pdbs, length=200):
    url = "http://www.rcsb.org/pdb/rest/search"
    query_text = f"""
<orgPdbCompositeQuery version="1.0">
 <queryRefinement>
   <queryRefinementLevel>0</queryRefinementLevel>
      <orgPdbQuery>
        <version>head</version>
        <queryType>org.pdb.query.simple.StructureIdQuery</queryType>
        <structureIdList>{' '.join(pdbs)}</structureIdList>
      </orgPdbQuery>
 </queryRefinement>
 <queryRefinement>
   <queryRefinementLevel>1</queryRefinementLevel>
       <orgPdbQuery>
        <version>head</version>
        <queryType>org.pdb.query.simple.SequenceLengthQuery</queryType>
        <v_sequence.chainLength.min>1</v_sequence.chainLength.min>
        <v_sequence.chainLength.max>{length}</v_sequence.chainLength.max>
      </orgPdbQuery>
  </queryRefinement>
</orgPdbCompositeQuery>
"""
    request = urllib.request.Request(url, data=query_text.encode())
    try:
        response = urllib.request.urlopen(request)
    except urllib.error.HTTPError as e:
        print(f"PDB error...")
        return None

    page = response.read()
    page = page.decode("utf-8").split()
    return page


In [8]:
small_pdbs = filter_chain_length(pdbs, length=200)

In [9]:
small_pdbs = [i.split(":")[0] for i in small_pdbs]

In [11]:
print(f"Filtered to {len(small_pdbs)} structures...")

Filtered to 101 structures...


Even though these PDB codes were listed in the validation-set on BindingDB, not all of these *exact* PDB codes have ligands.

In [12]:
def get_structures_with_ligands(pdbs):
    url = "http://www.rcsb.org/pdb/rest/search"
    query_text = f"""
<orgPdbCompositeQuery version="1.0">
 <queryRefinement>
  <queryRefinementLevel>0</queryRefinementLevel>
  <orgPdbQuery>
    <version>head</version>
    <queryType>org.pdb.query.simple.StructureIdQuery</queryType>
    <structureIdList>{" ".join(pdbs)}</structureIdList>
  </orgPdbQuery>
 </queryRefinement>
 <queryRefinement>
  <queryRefinementLevel>1</queryRefinementLevel>
  <conjunctionType>and</conjunctionType>
  <orgPdbQuery>
    <version>head</version>
    <queryType>org.pdb.query.simple.NoLigandQuery</queryType>
    <description>Ligand Search : Has free ligands=yes</description>
    <haveLigands>yes</haveLigands>
  </orgPdbQuery>
 </queryRefinement>
</orgPdbCompositeQuery>
"""
    request = urllib.request.Request(url, data=query_text.encode())
    try:
        response = urllib.request.urlopen(request)
    except urllib.error.HTTPError as e:
        print(f"PDB error...")
        return None

    page = response.read()
    page = page.decode("utf-8").split()
    return page


In [13]:
with_ligands = get_structures_with_ligands(small_pdbs)

In [15]:
print(f"Filtered to {len(with_ligands)} structures...")

Filtered to 97 structures...


## Combine the small protein structures with their ligand(s) into a single `pandas` DataFrame

In [16]:
def combine_structure_and_ligand(pdbs):
    url = f"""https://www.rcsb.org/pdb/rest/customReport.xml?pdbids={",".join(pdbs)}&customReportColumns=structureId,uniprotAcc,ligandName,ligandId,ligandMolecularWeight&service=wsfile&format=csv"""
    request = urllib.request.Request(url)
    try:
        response = urllib.request.urlopen(request)
    except urllib.error.HTTPError as e:
        print(f"PDB error...")
        return None

    page = response.read()
    return pd.read_csv(StringIO(page.decode("utf-8")), sep=",")


In [17]:
ligand_details = combine_structure_and_ligand(with_ligands)


In [18]:
ligand_details.dropna(subset=["ligandName"], inplace=True)


Many structures will report multiple "ligands": cosolvents, glycerol, ions, and a true small molecule ligand. We're just interested in structures with a small molecule ligand, so for each structure we'll sort the "ligands" by molecular weight and take the protein + ligand complex with the highest molecular weight.

In [19]:
structures_with_ligands = pd.DataFrame()
for structure in pd.unique(ligand_details["structureId"]):
    df = ligand_details[ligand_details["structureId"] == structure]
    winner = df.sort_values(by="ligandMolecularWeight", ascending=False).iloc[0]
    structures_with_ligands = structures_with_ligands.append(winner, ignore_index=True)

## Filter for complexes that do not have histidine in the binding site

In [21]:
def run_chimera(input_file="tmp/chimera.com"):

    sp.call(f"chimera {input_file}", shell=True)


In [22]:
def find_histidines(
    pdb, ligand, input_file="tmp/contacts.com", output_file="contacts.txt"
):
    with open(input_file, "w") as f:
        string = f"""
open {pdb}
select :{ligand} z<5
writesel {output_file}
stop
        """
        f.write(string)


def get_histidines(
    histidines=["HIS", "HSE", "HSD", "HID", "HIP", "HIE"], input_file="tmp/contacts.txt"
):
    nearby_residues = pd.read_csv(
        input_file, sep=" ", names=["Model", "Res", "ResID.Chain"]
    )

    total = sum(nearby_residues["Res"].isin(histidines))
    return total


In [23]:
good_pocket = dict()
for pdb, ligand in zip(
    structures_with_ligands["structureId"], structures_with_ligands["ligandId"]
):
    chimera_in = f"queries/{pdb}-his.com"
    path = "queries"
    chimera_out = f"{pdb}-his.txt"

    find_histidines(pdb, ligand, input_file=chimera_in, output_file=chimera_out)
    if not os.path.exists(os.path.join(path, chimera_out)):
        run_chimera(input_file=chimera_in)
    good_pocket[pdb] = get_histidines(input_file=os.path.join(path, chimera_out))


In [24]:
structures_with_histidines = pd.DataFrame.from_dict(
    good_pocket, orient="index", columns=["Histidines"]
)
structures_with_histidines["structureId"] = structures_with_histidines.index


In [25]:
structures = pd.merge(structures_with_ligands, structures_with_histidines)
structures = structures[structures.Histidines == 0]

In [26]:
print(f"Filtered to {len(structures)} structures...")

Filtered to 95 structures...


## Filter for complexes that don't have an unbound metal ion

In [27]:
def find_metals_not_in_ligand(
    pdb,
    ligand,
    disallowed_mask=":He | :Li | :Be | :B | :F | :Ne | :Na | :Mg | :Al | :Si | :Cl | :Ar | :K | :Ca | :Sc | :Ti | :V | :Cr | :Mn | :Fe | :Co | :Ni | :Cu | :Zn | :Ga | :Ge | :As | :Se | :Br | :Kr | :Rb | :Sr | :Y | :Zr | :Nb | :Mo | :Tc | :Ru | :Rh | :Pd | :Ag | :Cd | :In",
    input_file="tmp/metals.com",
    output_file="metals.txt",
):
    with open(input_file, "w") as f:
        string = f"""
open {pdb}
select {disallowed_mask} & ~:{ligand}
writesel {output_file}
stop
        """
        f.write(string)


def get_metals_not_in_ligand(input_file):
    nearby_residues = pd.read_csv(
        input_file, sep=" ", names=["Model", "Res", "ResID.Chain"]
    )

    total = len(nearby_residues["Res"])
    return total


In [28]:
metals = dict()
for pdb, ligand in zip(structures["structureId"],
                       structures["ligandId"]):

    chimera_in = f"queries/{pdb}-metal.com"
    path = "queries"
    chimera_out = f"{pdb}-metal.txt"

    
    find_metals_not_in_ligand(pdb, ligand, input_file=chimera_in, output_file=chimera_out)
    if not os.path.exists(os.path.join(path, chimera_out)):
        run_chimera(input_file=chimera_in)
    metals[pdb] = get_metals_not_in_ligand(input_file=os.path.join(path, chimera_out))

In [29]:
structures_with_metals = pd.DataFrame.from_dict(
    metals, orient="index", columns=["Metals"]
)
structures_with_metals["structureId"] = structures_with_metals.index


In [30]:
structures = pd.merge(structures, structures_with_metals)
structures = structures[structures.Metals == 0]

In [31]:
print(f"Filtered to {len(structures)} structures...")

Filtered to 60 structures...


In [32]:
structures.head()

,chainId,ligandId,ligandMolecularWeight,ligandName,structureId,uniprotAcc,Histidines,Metals
0,H,MID,521.63,1-[N-(naphthalen-2-ylsulfonyl)glycyl-4-carbami...,1ETS,P00735,0,0
3,B,YZ9,234.21,7-HYDROXY-2-OXO-CHROMENE-3-CARBOXYLIC ACID ETH...,1GCZ,P14174,0,0
6,A,MAO,339.35,5'-DEOXY-5'-[N-METHYL-N-(2-AMINOOXYETHYL) AMIN...,1I72,P17707,0,0
7,A,C2P,323.20,CYTIDINE-2'-MONOPHOSPHATE,1JVU,P61823,0,0
8,A,MGP,538.22,7-METHYL-GUANOSINE-5'-TRIPHOSPHATE,1L8B,P63073,0,0


In [33]:
def chain_length(pdbs):
    url = f"""https://www.rcsb.org/pdb/rest/customReport.xml?pdbids={",".join(pdbs)}&customReportColumns=structureId,chainLength&service=wsfile&format=csv"""
    request = urllib.request.Request(url)
    try:
        response = urllib.request.urlopen(request)
    except urllib.error.HTTPError as e:
        print(f"PDB error...")
        return None

    page = response.read()
    return pd.read_csv(StringIO(page.decode("utf-8")), sep=",")


## Filter to make sure the chain the ligand is part of is under the threshold

In [34]:
queries = [".".join([pdb, chain]) for pdb, chain in zip(structures["structureId"], structures["chainId"])]

In [35]:
chain_lengths = chain_length(queries)

In [36]:
structures = pd.merge(structures, chain_lengths)

In [41]:
structures[structures.chainLength < 200].sample(10)

,chainId,ligandId,ligandMolecularWeight,ligandName,structureId,uniprotAcc,Histidines,Metals,chainLength
19,A,UA3,324.18,URACIL ARABINOSE-3'-PHOSPHATE,1W4O,P61823,0,0,124
14,A,853,585.65,5-[2-ACETYLAMINO-2-(1-BIPHENYL-4-YLMETHYL-2-OX...,1O48,P12931,0,0,108
15,A,493,637.62,{4-[2-ACETYLAMINO-2-(1-BIPHENYL-4-YLMETHYL-2-O...,1O49,P12931,0,0,108
51,C,LPO,663.03,"(3S,4R)-N-[2-chloro-5-(2-methoxyethyl)benzyl]-...",3OAD,P00797,0,0,166
4,A,MGP,538.22,7-METHYL-GUANOSINE-5'-TRIPHOSPHATE,1L8B,P63073,0,0,190
18,B,PAX,809.36,5'-PHOSPHOTHYMIDINE (3'-5')-PYROPHOSPHATE ADEN...,1U1B,P61823,0,0,124
17,A,C3P,323.20,CYTIDINE-3'-MONOPHOSPHATE,1RPF,P61823,0,0,124
11,A,852,629.66,2-{4-[2-ACETYLAMINO-2-(1-BIPHENYL-4-YLMETHYL-2...,1O44,P12931,0,0,108
36,A,TXS,295.33,"1-(2,5-dideoxy-5-pyrrolidin-1-yl-beta-L-erythr...",3D8Z,P61823,0,0,124
1,B,YZ9,234.21,7-HYDROXY-2-OXO-CHROMENE-3-CARBOXYLIC ACID ETH...,1GCZ,P14174,0,0,122


In [40]:
structures[structures.chainLength < 200].to_csv("structures.csv")

In [95]:
def render_pymol(
    pdb,
    ligand,
    input_file="tmp/snapshot.com",
):
    with open(input_file, "w") as f:
        string = f"""\
fetch {pdb}, async=0

hide
show cartoon
show sticks, resn {ligand}

color green, resn {ligand}
color atomic, resn and not elem C

color grey80, polymer
show surface, polymer
set transparency, 0.3

bg_color white

center resn {ligand}
zoom resn {ligand}, 30

set cartoon_fancy_helices
png {pdb}.png, dpi=150

        """
        f.write(string)


In [97]:
for pdb, ligand in zip(structures["structureId"],
                       structures["ligandId"]):

    pymol_in = f"queries/{pdb}-pymol.pml"
    path = "queries"
    
    if not os.path.exists(os.path.join(path, pymol_in)):
        render_pymol(pdb=pdb,
                     ligand=ligand,
                     input_file=pymol_in)
        sp.call(f"pymol -cq {pymol_in}", shell=True)